# Pipeline 3: amplicon clustering with DADA2

Of the various OTU clustering amplicon pipelines out there (many! plenty of them good!), this one looked thoughtful and up-to-date to me. 

Description from the softwar help section:
"Accurate, high-resolution sample inference from
amplicon sequencing data
Description:            The dada2 package infers exact amplicon
sequence variants (ASVs) from high-throughput
amplicon sequencing data, replacing the coarser
and less accurate OTU clustering approach. The
dada2 pipeline takes as input demultiplexed
fastq files, and outputs the sequence variants
and their sample-wise abundances after removing
substitution and chimera errors. Taxonomic
classification is available via a native
implementation of the RDP naive Bayesian
classifier, and species-level assignment to 16S
rRNA gene fragments by exact matching."

<br>
I'll base my work flow here on software creators' tutorial pipeline:<br>
https://benjjneb.github.io/dada2/tutorial.html<br>
I don't plan to exactly do everything they do (our data set & goals are different), but largely working off their recommendations. <br>
Also useful and part of this work is the dada2 ITS specific work flow (keeping both open throughout!)<br>
https://benjjneb.github.io/dada2/ITS_workflow.html

jeezy pete R code makes me nauseous.

Work log
 - 2/8/2021 got this code started, spent more time with dada manual, figured some stuff out, planned some steps
 - 2/9/2021 trying to make steady progress, code up one step (cutadapt)

## -1. Set up tools and figure some stuff out

In [29]:
#Load python packages
import subprocess
import os
from Bio import Seq
from tqdm import tqdm

In [25]:
#Set a variable for project paths
basepath = "/Users/jessie/Dropbox/jessie/russula_illumina/"

### Run an R script from here?
I like using python for sequence manipulation because I have more memorized, but it's a little funny because this pipeline's main tools will be in R. Run as R scripts through command line here. Long term, might be better to do this as an R markdown. Trying first here to get my feet wet, then can reconsider. Another way to handle it would be to run analyses in python and pipeline in R in separate parallel notebooks. Upside is that writing R scripts for now will make it easy to convert to R markdown if needed (code the same).  

First up, learn how to run an R script. Here's an example:
https://stackoverflow.com/questions/18306362/run-r-script-from-command-line


In [19]:
#This is also awkward, but write their example R script to file here 
#(easier to write separately, but curious how how I can get content all in this document)
testscript = "sayHello <- function(){\n\
                print('hello')\n\
                }\n\
                sayHello()\n"

with open("testscript.r", "w") as outhandle:
    outhandle.write(testscript)

#Try running it
#Save stdout to a file along the way
with open("test_output.txt", "w") as stdouthandle:
    
    #Compose command
    cmd = f"Rscript testscript.r"
    print(cmd+"\n")

    #Run command
    subprocess.call(cmd, stdin=None, stdout=stdouthandle, stderr=subprocess.STDOUT, shell=True)        

#See results
with open("test_output.txt", "r") as check:
    for line in check:
        print(line.strip())


Rscript testscript.r

[1] "hello"


Sweet, that part's not hard. 

## Install dada2
https://benjjneb.github.io/dada2/dada-installation.html
Requires at least R 4.0.0.

### Update R
Is my R up to date? In terminal, launch R (type R). 3.6.0. 
No. Update!
To update, I'm downloading latest from here:
https://cloud.r-project.org/bin/macosx/
my mac is currently running 10.13.6, which is fine for latest R, which is 4.0.3.
click to launch installer. 
Check in fresh terminal window, R, now 4.0.3.

### Run dada2 installation R code
following:
https://benjjneb.github.io/dada2/dada-installation.html
except that it references previous bioconductor (changed for latest).
https://www.bioconductor.org/install/
and i needed to set a a cran mirror when running outside of R studio
https://stackoverflow.com/questions/33969024/install-packages-fails-in-knitr-document-trying-to-use-cran-without-setting-a

Saved install commands to: install_dada2.r


In [20]:
#Here's what it looks like
with open("install_dada2.r", "r") as handle:
    for line in handle:
        print(line.strip())



if (!requireNamespace("BiocManager", quietly = TRUE))
install.packages("BiocManager", repos = "http://cran.us.r-project.org")
BiocManager::install(version = "3.12")

if (!requireNamespace("BiocManager", quietly = TRUE))
install.packages("BiocManager", repos = "http://cran.us.r-project.org")
BiocManager::install("dada2", version = "3.12")


In [17]:
#Run install script - slow, lots of stuff in Bioconductor
#Save stdout to a file along the way
with open("install_dada2_stdout.txt", "w") as stdouthandle:
    #Compose command
    cmd = f"Rscript install_dada2.r"
    print(cmd)

    #Run command
    subprocess.call(cmd, stdin=None, stdout=stdouthandle, stderr=subprocess.STDOUT, shell=True)        

#See results
with open("install_dada2_stdout.txt", "r") as check:
    for line in check:
        print(line.strip())

Rscript install_dada2.r
trying URL 'http://cran.us.r-project.org/bin/macosx/contrib/4.0/BiocManager_1.30.10.tgz'

Content type 'application/x-gzip' length 94809 bytes (92 KB)


downloaded 92 KB





The downloaded binary packages are in

	/var/folders/8t/5vjtqgs165v6456qttd8glhw0000gn/T//RtmpwShuoT/downloaded_packages

Bioconductor version 3.12 (BiocManager 1.30.10), R 4.0.3 (2020-10-10)

Installing package(s) 'BiocVersion'

trying URL 'https://bioconductor.org/packages/3.12/bioc/bin/macosx/contrib/4.0/BiocVersion_3.12.0.tgz'

Content type 'application/x-gzip' length 5527 bytes


downloaded 5527 bytes





The downloaded binary packages are in

	/var/folders/8t/5vjtqgs165v6456qttd8glhw0000gn/T//RtmpwShuoT/downloaded_packages

Old packages: 'boot', 'class', 'codetools', 'foreign', 'KernSmooth', 'Matrix',

  'nlme', 'nnet', 'spatial'

Bioconductor version 3.12 (BiocManager 1.30.10), R 4.0.3 (2020-10-10)

Installing package(s) 'dada2'

also installing the dependencies ‘prettyunits’, ‘rpro

In [22]:
#Check dada2 installation with another little script

#Here's what it looks like
with open("check_dada2_installation.r", "r") as handle:
    for line in handle:
        print(line.strip())
        
#Save stdout to a file along the way
with open("check_dada2_installation_stdout.txt", "w") as stdouthandle:
    #Compose command
    cmd = f"Rscript check_dada2_installation.r"
    print(cmd)

    #Run command
    subprocess.call(cmd, stdin=None, stdout=stdouthandle, stderr=subprocess.STDOUT, shell=True)        

#See results
with open("check_dada2_installation_stdout.txt", "r") as check:
    for line in check:
        print(line.strip())

library("dada2")
sessionInfo()
help(package="dada2")
Rscript check_dada2_installation.r
Loading required package: Rcpp
R version 4.0.3 (2020-10-10)
Platform: x86_64-apple-darwin17.0 (64-bit)
Running under: macOS High Sierra 10.13.6

Matrix products: default
BLAS:   /Library/Frameworks/R.framework/Versions/4.0/Resources/lib/libRblas.dylib
LAPACK: /Library/Frameworks/R.framework/Versions/4.0/Resources/lib/libRlapack.dylib

locale:
[1] en_US.UTF-8/en_US.UTF-8/en_US.UTF-8/C/en_US.UTF-8/en_US.UTF-8

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base

other attached packages:
[1] dada2_1.18.0 Rcpp_1.0.6

loaded via a namespace (and not attached):
[1] plyr_1.8.6                  RColorBrewer_1.1-2
[3] pillar_1.4.7                compiler_4.0.3
[5] GenomeInfoDb_1.26.2         XVector_0.30.0
[7] MatrixGenerics_1.2.1        bitops_1.0-6
[9] tools_4.0.3                 zlibbioc_1.36.0
[11] lattice_0.20-41             lifecycle_0.2.0
[13] tibble_3.0.6     

So we have dada2 version 1.18.0, excellent. 

## How to get data ready for pipeline

From Tutorial:
"This workflow assumes that your sequencing data meets certain criteria:<br>
 - Samples have been demultiplexed, i.e. split into individual per-sample fastq files.
 - Non-biological nucleotides have been removed, e.g. primers, adapters, linkers, etc.
 - If paired-end sequencing data, the forward and reverse fastq files contain reads in matched order.<br>

If these criteria are not true for your data (are you sure there aren’t any primers hanging around?) you need to remedy those issues before beginning this workflow. See the FAQ for recommendations for some common issues."

From FAQ: cutadapt and trimmomatic suggested for more complicated primers like heterogeneity spacers (like ours). 
For this task I like cutadapt better, Trimmomatic has a clunky input and less control over processing.

The pipeline includes its own quality filtering later, so we just need to nix primers and adapters first. The dada2 ITS pipeline example includes removal of all primers in all orientations as a failsafe, but cutadapt manual favors  a targeted approach (use all of what you know about sequences to guide trimming - some of those orientations are way more likely than others in different parts of the reads). 

Because 3' adapter readthrough artifacts are outside of primers, and cutadapt's main trim settings remove distal material to the targetd sequence, we can just run a simple primer removal with cutadapt and also take care of the smattering of those other errors. This would entail removing f primers from f and r primers from r reads (easy ones that are always present at start of read), but also the reverse complement of the reverse primer from forward reads and the reverse complement of forward primer from reverse reads, to account for occasional short-insert read through.  (that second part is what covers adapter readthrough artifacts, but is worthwhile on its own, too, since we want all primers gone and this is the second most likely way to get primers in the data).


## 1. Remove primers (and adapters) from reads
cutadapt<br>
A read<br>
--------><br>
definitely has its own primer (f/f, r/r)<br>
->-------<br>
occasionally (short insert by nature or mistake) has complement of other primer, too<br>
->------<-<br>

Remove both of these cases. 

Cutadapt (default) will also remove upstream material, so we don't have to account for spacers (they are distal), or Illumina adapters (3' overhangs distal to 3' primer cut).<br>

Cutadapt option choices:<br>
https://cutadapt.readthedocs.io/en/stable/guide.html<br>
-o f output<br>
-p r output. run both pairs at once (makes sure if one of a pair tossed, other one is too, keeps dataset as all pairs)<br>

Trimming parameters: <br>
-g = remove given primer from 5' end of f read (forward primer) <br>
-G = remove given primer from 5' end of r read (reverse primer) <br>
-a = remove given primer from 3' end of f read (complement of reverse primer) <br>
-A = remove given primer from 3' end of r read (complement of forward primer) <br>

Post-processing: <br>
-m = minimum length of sequence, discard reads shorter than value after trimming (200)<br>
<br>
Not saving --info-file - no time to get into details and want to save space. 

There's some complicated stuff about running cutadapt with multiple arguments and pair mode (finds best only, doesn't save info files, requires both adapters). So, to get full information and control, run serially. First remove f primer from f reads 5' end. Then, remove r primer from r reads 5' end. Then, remove complements from 3' ends. That way we can be sure all of the steps happen and make sure all the results data is produced. Slightly annoying. 


In [41]:
#Make a directory for files without primers, cutadapt pair mode results:
if not os.path.exists("1_remove_primers/pair_mode"):
    os.system("mkdir 1_remove_primers/pair_mode")


In [42]:
#Locate raw input files (includes split replicates)
allrawdir = os.listdir(basepath+'Russula_BC')
forward_fastqs = []
reverse_fastqs = []
for file in allrawdir:
    if file.endswith("_R1.fastq"):
        forward_fastqs.append(basepath+'Russula_BC/'+file)
    elif file.endswith("_R2.fastq"):
        reverse_fastqs.append(basepath+'Russula_BC/'+file)

#Sort to make sure pairs are in the same order in these lists
forward_fastqs = sorted(forward_fastqs)
reverse_fastqs = sorted(reverse_fastqs)

#Report back on findings (and read to make sure sensible)
print(f"Found {len(forward_fastqs)} forward files with {len(set(forward_fastqs))} unique names, like {forward_fastqs[0]}")
print(f"Found {len(reverse_fastqs)} forward files with {len(set(reverse_fastqs))} unique names, like {reverse_fastqs[0]}")


Found 196 forward files with 196 unique names, like /Users/jessie/Dropbox/jessie/russula_illumina/Russula_BC/DBG_016111_R1.fastq
Found 196 forward files with 196 unique names, like /Users/jessie/Dropbox/jessie/russula_illumina/Russula_BC/DBG_016111_R2.fastq


In [43]:
#Gather primer sequences

#Forward primer - without spacer (on 5' end, distal to cuts), with R for degenerate base (cutadapt fine with that)
forward_primer = Seq.Seq("GTGARTCATCGAATCTTTG")#converting to sequence object for handy complementing
forward_primer_complement = forward_primer.reverse_complement()
#Reverse primer - without spacer (on 5' end) and without 5 Ns for barcode region (between spacer and primer). 
reverse_primer = Seq.Seq("TCCTCCGCTTATTGATATGC")
#Could run with precise primers (have mapping handy), but actually not doing much because on distal end of cuts.
#Leaving Ns/barcodes out because they inflate proportion of matches - more even handling compared to f primer this way.
reverse_primer_complement = reverse_primer.reverse_complement()


In [44]:
#Run cutadapt in pair mode

#Save stdout to a file along the way - cutadapt prints lots of reporting information here
with open("1_cutadapt_stdout_primer_removal_pair_mode.txt", "w") as stdouthandle:
    
    #Loop over pairs
    for i in tqdm(range(len(forward_fastqs))):
        
        #Grab a specimen's pair of data files (they are ordered)
        ffile = forward_fastqs[i]
        rfile = reverse_fastqs[i]
        
        #Make up names for output files
        fbasename = ffile.split("/")[-1].split(".")[0]
        rbasename = rfile.split("/")[-1].split(".")[0]
        foutfastq = "1_remove_primers/pair_mode/"+fbasename+"_pair_mode_primers_trimmed.fastq"
        routfastq = "1_remove_primers/pair_mode/"+rbasename+"_pair_mode_primers_trimmed.fastq"
        info = foutfastq.split("_R")[0]+"_pair_mode_cutadapt_info_file.txt"

        #Compose command
        #Converting primer sequences back to strings
        cmd = f"cutadapt -g {str(forward_primer)} -G {str(reverse_primer)} \
                -a {str(reverse_primer_complement)} -A {str(forward_primer_complement)} \
                -m 200 --info-file {info} -o {foutfastq} -p {routfastq} {ffile} {rfile}"
        
        print(cmd+"\n")
        
        #Run command
        subprocess.call(cmd, stdin=None, stdout=stdouthandle, stderr=subprocess.STDOUT, shell=True)        




  0%|          | 0/196 [00:00<?, ?it/s]

cutadapt -g GTGARTCATCGAATCTTTG -G TCCTCCGCTTATTGATATGC                 -a GCATATCAATAAGCGGAGGA -A CAAAGATTCGATGAYTCAC                 -m 200 --info-file 1_remove_primers/pair_mode/DBG_016111_pair_mode_cutadapt_info_file.txt -o 1_remove_primers/pair_mode/DBG_016111_R1_pair_mode_primers_trimmed.fastq -p 1_remove_primers/pair_mode/DBG_016111_R2_pair_mode_primers_trimmed.fastq /Users/jessie/Dropbox/jessie/russula_illumina/Russula_BC/DBG_016111_R1.fastq /Users/jessie/Dropbox/jessie/russula_illumina/Russula_BC/DBG_016111_R2.fastq



  1%|          | 1/196 [00:01<05:31,  1.70s/it]

cutadapt -g GTGARTCATCGAATCTTTG -G TCCTCCGCTTATTGATATGC                 -a GCATATCAATAAGCGGAGGA -A CAAAGATTCGATGAYTCAC                 -m 200 --info-file 1_remove_primers/pair_mode/DBG_016192_pair_mode_cutadapt_info_file.txt -o 1_remove_primers/pair_mode/DBG_016192_R1_pair_mode_primers_trimmed.fastq -p 1_remove_primers/pair_mode/DBG_016192_R2_pair_mode_primers_trimmed.fastq /Users/jessie/Dropbox/jessie/russula_illumina/Russula_BC/DBG_016192_R1.fastq /Users/jessie/Dropbox/jessie/russula_illumina/Russula_BC/DBG_016192_R2.fastq



  1%|          | 2/196 [00:03<05:15,  1.63s/it]

...

 99%|█████████▉| 194/196 [05:18<00:04,  2.47s/it]

cutadapt -g GTGARTCATCGAATCTTTG -G TCCTCCGCTTATTGATATGC                 -a GCATATCAATAAGCGGAGGA -A CAAAGATTCGATGAYTCAC                 -m 200 --info-file 1_remove_primers/pair_mode/DBG_030440_pair_mode_cutadapt_info_file.txt -o 1_remove_primers/pair_mode/DBG_030440_R1_pair_mode_primers_trimmed.fastq -p 1_remove_primers/pair_mode/DBG_030440_R2_pair_mode_primers_trimmed.fastq /Users/jessie/Dropbox/jessie/russula_illumina/Russula_BC/DBG_030440_R1.fastq /Users/jessie/Dropbox/jessie/russula_illumina/Russula_BC/DBG_030440_R2.fastq



 99%|█████████▉| 195/196 [05:22<00:02,  2.91s/it]

cutadapt -g GTGARTCATCGAATCTTTG -G TCCTCCGCTTATTGATATGC                 -a GCATATCAATAAGCGGAGGA -A CAAAGATTCGATGAYTCAC                 -m 200 --info-file 1_remove_primers/pair_mode/DBG_030462_pair_mode_cutadapt_info_file.txt -o 1_remove_primers/pair_mode/DBG_030462_R1_pair_mode_primers_trimmed.fastq -p 1_remove_primers/pair_mode/DBG_030462_R2_pair_mode_primers_trimmed.fastq /Users/jessie/Dropbox/jessie/russula_illumina/Russula_BC/DBG_030462_R1.fastq /Users/jessie/Dropbox/jessie/russula_illumina/Russula_BC/DBG_030462_R2.fastq



100%|██████████| 196/196 [05:25<00:00,  2.88s/it]


Skim standard output file. Here's one sample's example in output. <br>

This is cutadapt 3.0 with Python 3.7.1<br>
Command line parameters: -g GTGARTCATCGAATCTTTG -G TCCTCCGCTTATTGATATGC -a GCATATCAATAAGCGGAGGA -A CAAAGATTCGATGAYTCAC -m 200 -o 1_remove_primers/DBG_023286A_R1_primers_trimmed.fastq -p 1_remove_primers/DBG_023286A_R2_primers_trimmed.fastq /Users/jessie/Dropbox/jessie/russula_illumina/Russula_BC/DBG_023286A_R1.fastq /Users/jessie/Dropbox/jessie/russula_illumina/Russula_BC/DBG_023286A_R2.fastq<br>
Processing reads on 1 core in paired-end mode ...<br>
Finished in 0.93 s (52 µs/read; 1.14 M reads/minute).<br>
<br>
=== Summary ===<br>
<br>
Total read pairs processed:             17,768<br>
  Read 1 with adapter:                  17,768 (100.0%)<br>
  Read 2 with adapter:                  17,768 (100.0%)<br>
Pairs that were too short:                  34 (0.2%)<br>
Pairs written (passing filters):        17,734 (99.8%)<br>
<br>
Total basepairs processed:     8,919,536 bp<br>
  Read 1:     4,459,768 bp<br>
  Read 2:     4,459,768 bp<br>
Total written (filtered):      8,092,692 bp (90.7%)<br>
  Read 1:     4,101,892 bp<br>
  Read 2:     3,990,800 bp<br>
<br>
=== First read: Adapter 1 ===<br>
<br>
Sequence: GTGARTCATCGAATCTTTG; Type: regular 5'; Length: 19; Trimmed: 17734 times<br>
<br>
No. of allowed errors:<br>
1-9 bp: 0; 10-19 bp: 1<br>
<br>
Overview of removed sequences<br>
length	count	expect	max.err	error counts<br>
19	11536	0.0	1	11536<br>
21	6198	0.0	1	6198<br>

<br>
=== First read: Adapter 2 ===<br>
<br>
Sequence: GCATATCAATAAGCGGAGGA; Type: regular 3'; Length: 20; Trimmed: 34 times<br>
<br>
No. of allowed errors:<br>
1-9 bp: 0; 10-19 bp: 1; 20 bp: 2<br>
<br>
Bases preceding removed adapters:<br>
  A: 20.6%<br>
  C: 50.0%<br>
  G: 5.9%<br>
  T: 23.5%<br>
  none/other: 0.0%<br>
<br>
Overview of removed sequences<br>
length	count	expect	max.err	error counts<br>
55	1	0.0	2	1<br>
193	1	0.0	2	1<br>
195	6	0.0	2	6<br>
197	1	0.0	2	1<br>
212	1	0.0	2	1<br>
220	2	0.0	2	2<br>
223	1	0.0	2	1<br>
224	11	0.0	2	11<br>
225	3	0.0	2	3<br>
226	3	0.0	2	3<br>
227	1	0.0	2	1<br>
232	1	0.0	2	1<br>
233	2	0.0	2	2<br>
<br>

=== Second read: Adapter 3 ===<br>
<br>
Sequence: TCCTCCGCTTATTGATATGC; Type: regular 5'; Length: 20; Trimmed: 17768 times<br>
<br>
No. of allowed errors:<br>
1-9 bp: 0; 10-19 bp: 1; 20 bp: 2<br>
<br>
Overview of removed sequences<br>
length	count	expect	max.err	error counts<br>
25	9211	0.0	2	9211<br>
27	8557	0.0	2	8557<br>
<br>

=== Second read: Adapter 4 ===<br>
<br>
Sequence: CAAAGATTCGATGAYTCAC; Type: regular 3'; Length: 19; Trimmed: 0 times<br>

Skimming the results, the second read adapter 4 is never found. This is the complement of the forward primer, searched on the reverse read. Sequence looks right. Opposite case of primer on 3' end was found sometimes for other end, and I would have thought this one would be comparably frequent (those same fragments, for example). Maybe the lower quality on reverse read has something to do with it? Something else wrong with how I did it? Maybe if F read is tossed it doesn't check R read, so it never makes it to this point?

Some samples in output have this warning:
"WARNING:
    One or more of your adapter sequences may be incomplete.
    Please see the detailed output above."
Reading manual, this is what happens when a conserved base is adjacent to the trimmed sequence

For example, this is a cut of the reverse complement of the reverse primer from the 3' end of a forward read set. The base C is always before the adapter sequence (5' end). The program is suggesting we add a C. 
"=== First read: Adapter 2 ===

Sequence: GCATATCAATAAGCGGAGGA; Type: regular 3'; Length: 20; Trimmed: 28 times

No. of allowed errors:
1-9 bp: 0; 10-19 bp: 1; 20 bp: 2

Bases preceding removed adapters:
  A: 0.0%
  C: 100.0%
  G: 0.0%
  T: 0.0%
  none/other: 0.0%
WARNING:
    The adapter is preceded by "C" extremely often.
    The provided adapter sequence could be incomplete at its 5' end.

Overview of removed sequences
length	count	expect	max.err	error counts
213	1	0.0	2	1
223	10	0.0	2	8 2
225	16	0.0	2	16
227	1	0.0	2	1"

pulling an overlapped sequence fragment (DBG_030462_overlapped_m30p0.fastq) without 3' adapters but with primers intact:

GTGAATCATCGAATCTTTG #starts with f primer TACGCACCTTGCGCCCCTTGGCATTCCGAGGGGCACACCCGTTTGAGTGTCGTGAAATCATCAAAACCTTCTCTTTGACCCTCTTTGGTCGGGAAAAGGATTTTTGGACTTGGAGGTTCCATGCTCGCCTTCACTTTTGAAAGTGAGCTCCTCTCAAATGAATTAGTGGGGTCCGCTTTGCTGGTCCTTGTCGTGATAAGATGTTTCTACGTTTTGGATTTAGCACTGTCCCTTGGATGCCTGCTCCTAACTGTCTCACGGACAACGATGGTGCTCTGGTCACTGCCATCTACATTGGCGGGAGGCTGGACCCACAAAAGAAAACCTTGACCTCAAATCGGGTGAGACTACCCGCTGAACTTAA #fragment GCATATCAATAAGCGGAGGA #reverse primer complement CTCGT #barcode AG #spacer

That doesn't have a C at 5' end for 3' primer. weird. Ok, re run with --info-file since a few things are fishy (replacing above code). And not all samples have this warning. weird. 

Another wrinkle:
"The --info-file, --rest-file and --wildcard-file options write out information only from the first read."

To get full info I could run f/r separately then post-process to recover pairs (not very hard). Would be helpful to fully describe issues. Leave for now, then run separate version. Ooph. annoying. kinda hard to do 5' and 3' cutadapts in one go - better to run serially. annoying. 


In [ ]:
#Make a directory for files without primers, cutadapt run separately on f and r:
if not os.path.exists("1_remove_primers/f_r_separate"):
    os.system("mkdir 1_remove_primers/f_r_separate")


In [45]:
#Run cutadapt separately on f and r reads to get more information about process

#Save stdout to a file along the way - cutadapt prints lots of reporting information here
with open("1_cutadapt_stdout_primer_removal_forward_reads_f_r_separate.txt", "w") as stdouthandle:
    
    #Loop over pairs
    for i in tqdm(range(len(forward_fastqs))):
        
        #Grab a specimen's pair of data files (they are ordered)
        ffile = forward_fastqs[i]
        
        #Make up names for output files
        fbasename = ffile.split("/")[-1].split(".")[0]
        foutfastq = "1_remove_primers/f_r_separate/"+fbasename+"_f_r_separate_primers_trimmed.fastq"        
        info1 = foutfastq.split("_R")[0]+"_forward_reads_f_r_separate_cutadapt_info_file.txt"

        #Compose commands
        #Converting primer sequences back to strings
        cmd1 = f"cutadapt -g {str(forward_primer)} -a {str(reverse_primer_complement)} \
                -m 200 --info-file {info1} -o {foutfastq} {ffile}"

        print(cmd1+"\n")
        
        #Run command
        #subprocess.call(cmd1, stdin=None, stdout=stdouthandle, stderr=subprocess.STDOUT, shell=True)        

        
#Save stdout to a file along the way - cutadapt prints lots of reporting information here
with open("1_cutadapt_stdout_primer_removal_reverse_reads_f_r_separate.txt", "w") as stdouthandle:
    
    #Loop over pairs
    for i in tqdm(range(len(reverse_fastqs))):
        
        #Grab a specimen's pair of data files (they are ordered)
        rfile = reverse_fastqs[i]
        
        #Make up names for output files
        rbasename = rfile.split("/")[-1].split(".")[0]
        routfastq = "1_remove_primers/f_r_separate/"+rbasename+"_f_r_separate_primers_trimmed.fastq"
        
        info2 = foutfastq.split("_R")[0]+"_reverse_reads_f_r_separate_cutadapt_info_file.txt"

        #Compose command
        #Converting primer sequences back to strings
        cmd2 = f"cutadapt -g {str(reverse_primer)} -a {str(forward_primer_complement)} \
        -m 200 --info-file {info2} -o {routfastq} {rfile}"
        
        #Check command before running
        print(cmd2+"\n")

        #Run command
        #subprocess.call(cmd2, stdin=None, stdout=stdouthandle, stderr=subprocess.STDOUT, shell=True)        


100%|██████████| 196/196 [00:00<00:00, 22290.77it/s]

cutadapt -g GTGARTCATCGAATCTTTG -a GCATATCAATAAGCGGAGGA                 -m 200 --info-file 1_remove_primers/f_r_separate/DBG_016111_forward_reads_f_r_separate_cutadapt_info_file.txt -o 1_remove_primers/f_r_separate/DBG_016111_R1_f_r_separate_primers_trimmed.fastq /Users/jessie/Dropbox/jessie/russula_illumina/Russula_BC/DBG_016111_R1.fastq

cutadapt -g GTGARTCATCGAATCTTTG -a GCATATCAATAAGCGGAGGA                 -m 200 --info-file 1_remove_primers/f_r_separate/DBG_016192_forward_reads_f_r_separate_cutadapt_info_file.txt -o 1_remove_primers/f_r_separate/DBG_016192_R1_f_r_separate_primers_trimmed.fastq /Users/jessie/Dropbox/jessie/russula_illumina/Russula_BC/DBG_016192_R1.fastq

cutadapt -g GTGARTCATCGAATCTTTG -a GCATATCAATAAGCGGAGGA                 -m 200 --info-file 1_remove_primers/f_r_separate/DBG_016193_forward_reads_f_r_separate_cutadapt_info_file.txt -o 1_remove_primers/f_r_separate/DBG_016193_R1_f_r_separate_primers_trimmed.fastq /Users/jessie/Dropbox/jessie/russula_illumina/Russul